In [72]:
data = catalog.load("master_table")
serie_target = catalog.load("params:serie_target")
date_col = catalog.load("params:serie_period")
serie_id = catalog.load("params:series_level.columns")
serie_freq = catalog.load("params:serie_freq")
n_jobs = catalog.load("params:n_jobs")

2022-04-03 21:24:15,002 - kedro.io.data_catalog - INFO - Loading data from `master_table` (CSVDataSet)...
2022-04-03 21:24:16,603 - kedro.io.data_catalog - INFO - Loading data from `params:serie_target` (MemoryDataSet)...
2022-04-03 21:24:16,626 - kedro.io.data_catalog - INFO - Loading data from `params:serie_period` (MemoryDataSet)...
2022-04-03 21:24:16,627 - kedro.io.data_catalog - INFO - Loading data from `params:series_level.columns` (MemoryDataSet)...
2022-04-03 21:24:16,628 - kedro.io.data_catalog - INFO - Loading data from `params:serie_freq` (MemoryDataSet)...
2022-04-03 21:24:16,629 - kedro.io.data_catalog - INFO - Loading data from `params:n_jobs` (MemoryDataSet)...


# Baseline

In [80]:
from typing import List, Optional, Union

from tqdm import tqdm

def prepare_time_series(
    data: pd.DataFrame, 
    date_col: str, 
    serie_target: str, 
    serie_id: Union[str, List[str]],
    ) -> pd.DataFrame:
    """
    This node prepare time series, ensuring that all series have all periods 
    (adding duplicate periods and adding periods without observations) and 
    filling null values.

    Args:
        data: Dataframe with time series.
        date_col: Period column name.
        serie_target: Target column name.
        serie_id: Column or list of columns that identify series.

    Returns:
        Data with prepared time series
    """
    data["serie_id"] = list(map(str, zip(*[data[c] for c in serie_id])))
    data = data.groupby([date_col, "serie_id"]).sum()[serie_target].reset_index()
    tqdm.pandas()
    data = data.groupby("serie_id").progress_apply(lambda serie_data: _build_series(serie_data, serie_target, date_col))
    return data.reset_index()

def _build_series(
    serie_data: pd.DataFrame, 
    serie_target: str, 
    date_col: str) -> pd.DataFrame:
    """
    This function prepare a time series, ensuring that all series have all periods 
    (adding duplicate periods and periods without observations) and 
    filling null values.

    Args:
        serie_data: Dataframe with time series.
        serie_target: Target column name.
        date_col: Period column name.

    Returns:
        Data with prepared time serie
    """
    
    serie = serie_data.set_index(date_col)[[serie_target]]
    full_serie = serie.reindex(pd.Index(pd.date_range(serie.index.min(), serie.index.max()), name="date"))
    full_serie[serie_target] = _rolling_fill(full_serie[serie_target], n=2)
    return full_serie

In [81]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import pandas as pd
def _rolling_fill(
    data: pd.Series,
    n: int
) -> pd.Series:

    """
    Fills Na values with the mean of the nearest values.

    Args:
        data: Original Series.
        n: Window size.
    Return:
        Series with missing values filled. 
    """
    
    data[data < 0] = 0

    out = np.copy(data)
    w_size = n//2

    # Create sliding window view -> [[x[i]-1, x[i], x[i+1]] for i in range(x.shape)]
    rolling_mean = np.hstack((np.full(w_size, np.nan), out, np.full(w_size, np.nan)))
    axis = 0 if len(rolling_mean.shape) == 1 else 1
    rolling_mean = np.nanmean(sliding_window_view(rolling_mean, (n+1,), axis=axis), axis=1)
    # Get Mean e filling nan values
    out[np.isnan(out)] = rolling_mean[np.isnan(out)]
    out[np.isnan(out)] = 0

    return out

In [82]:
%time prepare_time_series(data, date_col, serie_target, serie_id)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1782/1782 [00:07<00:00, 243.40it/s]


CPU times: user 10.3 s, sys: 801 ms, total: 11.1 s
Wall time: 12.2 s


,serie_id,date,sales
0,"(1, 'AUTOMOTIVE')",2013-01-01,0.000000
1,"(1, 'AUTOMOTIVE')",2013-01-02,2.000000
2,"(1, 'AUTOMOTIVE')",2013-01-03,3.000000
3,"(1, 'AUTOMOTIVE')",2013-01-04,3.000000
4,"(1, 'AUTOMOTIVE')",2013-01-05,5.000000
...,...,...,...
3008011,"(9, 'SEAFOOD')",2017-08-11,23.831000
3008012,"(9, 'SEAFOOD')",2017-08-12,16.859001
3008013,"(9, 'SEAFOOD')",2017-08-13,20.000000
3008014,"(9, 'SEAFOOD')",2017-08-14,17.000000


# Optimization

In [56]:
dates = ["2013-01-01", "2013-01-02", "2013-01-03", "2013-01-04", "2013-01-05", "2013-01-07", # Serie "A"
         "2013-01-01", "2013-01-01", "2013-01-03", "2013-01-04", "2013-01-05", "2013-01-07", # Serie "B"
         "2013-01-01", "2013-01-02", "2013-01-03", "2013-01-04", "2013-01-07", # Serie "C"
         "2013-01-01", "2013-01-04", "2013-01-05", # Serie "D"
        ]
ids  = ["A", "A", "A", "A", "A", "A",
        "B", "B", "B", "B", "B", "B",
        "C", "C", "C", "C", "C",
        "D", "D", "D",
       ]
sales = [1, 1, 2, 3, 2, 1,
        1, 1, 2, 3, 2, 1,
        10, 5, 1, 10, 20,
        1, 2, 1,
       ]
test_data = pd.DataFrame({"date": dates, "serie_id": ids, "sales": sales})
test_data["date"] = pd.to_datetime(test_data.date)
test_data.sample(5)

,date,serie_id,sales
1,2013-01-02,A,1
5,2013-01-07,A,1
10,2013-01-05,B,2
2,2013-01-03,A,2
8,2013-01-03,B,2


In [57]:
series_data = test_data.pivot_table(columns="serie_id", values="sales", index="date")#
series_data = series_data.reindex(pd.Index(pd.date_range(series_data.index.min(), series_data.index.max()), name="date"))
series_data

serie_id,A,B,C,D
date,,,,
2013-01-01,1.0,1.0,10.0,1.0
2013-01-02,1.0,NaN,5.0,NaN
2013-01-03,2.0,2.0,1.0,NaN
2013-01-04,3.0,3.0,10.0,2.0
2013-01-05,2.0,2.0,NaN,1.0
2013-01-06,NaN,NaN,NaN,NaN
2013-01-07,1.0,1.0,20.0,NaN


In [108]:
def prepare_time_series(
    data: pd.DataFrame, 
    date_col: str, 
    serie_target: str, 
    serie_id: Union[str, List[str]],
    ) -> pd.DataFrame:
    """
    This node prepare time series, ensuring that all series have all periods 
    (adding duplicate periods and adding periods without observations) and 
    filling null values.

    Args:
        data: Dataframe with time series.
        date_col: Period column name.
        serie_target: Target column name.
        serie_id: Column or list of columns that identify series.

    Returns:
        Data with prepared time series
    """
    if "serie_id" not in data.columns:
        data["serie_id"] = list(map(str, zip(*[data[c] for c in serie_id])))
    series_data = data.pivot_table(columns="serie_id", values=serie_target, index=date_col)
    series_data = series_data.reindex(pd.Index(pd.date_range(series_data.index.min(), series_data.index.max()), name="date"))
    series_data = _rolling_mean(series_data)
    series_data = pd.melt(series_data, value_name=serie_target, ignore_index=False).reset_index()
    
    return series_data

def _rolling_mean(data: pd.DataFrame, n: int = 2) -> pd.DataFrame:
    """
    Fills Na values with the mean of the nearest values.

    Args:
        data: Original Series.
        n: Window size.
    Return:
        DataFrame with missing values filled. 
    """
    w_size = n//2
    out = np.copy(data)
    filler = np.full((w_size, out.shape[1]), np.nan)
    rolling_mean = np.vstack((filler, out, filler))
    rolling_mean = sliding_window_view(rolling_mean, n+1, 0)
    rolling_mean = np.nanmean(rolling_mean, axis=2)
    out[np.isnan(out)] = rolling_mean[np.isnan(out)]
    out[np.isnan(out)] = 0
    out_data = pd.DataFrame(data=out, columns=data.columns, index=data.index)
    return out_data 

In [109]:
series_data = prepare_time_series(test_data, date_col, serie_target, serie_id)
series_data

/tmp/ipykernel_13301/2582414766.py:45: RuntimeWarning: Mean of empty slice
  rolling_mean = np.nanmean(rolling_mean, axis=2)


,date,serie_id,sales
0,2013-01-01,A,1.0
1,2013-01-02,A,1.0
2,2013-01-03,A,2.0
3,2013-01-04,A,3.0
4,2013-01-05,A,2.0
5,2013-01-06,A,1.5
6,2013-01-07,A,1.0
7,2013-01-01,B,1.0
8,2013-01-02,B,1.5
9,2013-01-03,B,2.0


In [110]:
data = catalog.load("master_table")
serie_target = catalog.load("params:serie_target")
date_col = catalog.load("params:serie_period")
%time prepare_time_series(data, date_col, serie_target, serie_id)

2022-04-03 21:39:11,576 - kedro.io.data_catalog - INFO - Loading data from `master_table` (CSVDataSet)...
2022-04-03 21:39:13,171 - kedro.io.data_catalog - INFO - Loading data from `params:serie_target` (MemoryDataSet)...
2022-04-03 21:39:13,172 - kedro.io.data_catalog - INFO - Loading data from `params:serie_period` (MemoryDataSet)...


/home/matheus/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 3.44 s, sys: 255 ms, total: 3.69 s
Wall time: 3.69 s


,date,serie_id,sales
0,2013-01-01,"(1, 'AUTOMOTIVE')",0.000000
1,2013-01-02,"(1, 'AUTOMOTIVE')",2.000000
2,2013-01-03,"(1, 'AUTOMOTIVE')",3.000000
3,2013-01-04,"(1, 'AUTOMOTIVE')",3.000000
4,2013-01-05,"(1, 'AUTOMOTIVE')",5.000000
...,...,...,...
3008011,2017-08-11,"(9, 'SEAFOOD')",23.831000
3008012,2017-08-12,"(9, 'SEAFOOD')",16.859001
3008013,2017-08-13,"(9, 'SEAFOOD')",20.000000
3008014,2017-08-14,"(9, 'SEAFOOD')",17.000000


In [62]:
rolling_mean[np.isnan(series_data)]

array([ 1.5,  1. ,  2. , 10. ,  1.5,  1.5, 20. ,  1. ,  nan])

In [52]:
np.nanmean(rolling_mean, axis=2)

array([[ 1.        ,  1.        ,  7.5       ,  1.        ],
       [ 1.33333333,  1.5       ,  5.33333333,  1.        ],
       [ 2.        ,  2.5       ,  5.33333333,  2.        ],
       [ 2.33333333,  2.33333333,  5.5       ,  1.5       ],
       [ 2.5       ,  2.5       , 10.        ,  1.5       ],
       [ 1.5       ,  1.5       , 20.        ,  1.5       ],
       [ 1.        ,  1.        , 20.        ,  2.        ]])

In [43]:
sliding_window_view(x,3, 0)

array([[[ 0, 10, 20],
        [ 1, 11, 21],
        [ 2, 12, 22],
        [ 3, 13, 23]]])

In [45]:
sliding_window_view(rolling_mean, 3, 0)

array([[[nan,  1.,  1.],
        [nan,  1., nan],
        [nan, 10.,  5.],
        [nan,  1., nan]],

       [[ 1.,  1.,  2.],
        [ 1., nan,  2.],
        [10.,  5.,  1.],
        [ 1., nan, nan]],

       [[ 1.,  2.,  3.],
        [nan,  2.,  3.],
        [ 5.,  1., 10.],
        [nan, nan,  2.]],

       [[ 2.,  3.,  2.],
        [ 2.,  3.,  2.],
        [ 1., 10., nan],
        [nan,  2.,  1.]],

       [[ 3.,  2., nan],
        [ 3.,  2., nan],
        [10., nan, nan],
        [ 2.,  1., nan]],

       [[ 2., nan,  1.],
        [ 2., nan,  1.],
        [nan, nan, 20.],
        [ 1., nan,  2.]],

       [[nan,  1., nan],
        [nan,  1., nan],
        [nan, 20., nan],
        [nan,  2., nan]]])

In [ ]:
full_serie = serie.reindex(pd.Index(pd.date_range(serie.index.min(), serie.index.max()), name="date"))
full_serie[serie_target] = _rolling_fill(full_serie[serie_target], n=2)